# Semantic Kernel - Core Features 
Semantic Kernel is an open-source SDK designed for developers to easily integrate AI agents into their existing systems, facilitating automation and enhancing user experiences. This SDK bridges the gap between AI capabilities and practical application, making it straightforward to bring AI insights into actionable processes.

## Core Features
    0 AI Service Integration: 
It offers a uniform interface for connecting with various AI services like OpenAI, Azure OpenAI, HuggingFace, and Dall-e. This ensures developers can easily integrate different AI technologies without worrying about compatibility issues.

    1. Extending AI to Real-World Actions: 
Beyond mere integration, the SDK allows for the execution of native code and actions based on AI outputs, supporting complex workflows through input and output chaining.Unified Interface to AI services
Extend AI ablility to exteral world

## Example: A simple agent
In the following code, we are going to demostrate miniume steps of building an AI agent that follows user's instructions to complete task on a local system.

First we register an AI service. In this example it is an Azure OpenAI "GPT-4" deveployment.
Next, add TimePlugin and LightPlugin. The [TimePlugin](https://learn.microsoft.com/en-us/semantic-kernel/agents/plugins/out-of-the-box-plugins?tabs=Csharp) is an out-of-the-box plugin provided by Semantic Kernel, while LightPlugin is a plugin written in [native code](./plugins/LightPlugin.cs).
Under the hood, Semantic Kernel will do all the heavy lifting by constructing requests to the AI service, parsing the response, invoking local functions and chain the messages. 

In [21]:
#r "nuget: Microsoft.SemanticKernel, 1.3.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.3.1-alpha"
#!import config/Settings.cs 
#!import lib/Usings.cs
#!import lib/Utilities.cs
#!import plugins/LightPlugin.cs

Installed Packages Microsoft.SemanticKernel, 1.3.1 Microsoft.SemanticKernel.Plugins.Core, 1.3.1-alpha

In [2]:
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
var builder = Kernel.CreateBuilder();

//register ai serivces
builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);

//register plugins
#pragma warning disable SKEXP0050
builder.Plugins.AddFromType<TimePlugin>();
builder.Plugins.AddFromType<LightPlugin>();

var kernel = builder.Build();


## Lets see it in action. 
In the above code we have the kernel configured. Next step is actually send user request and see what happens.

We want to use our AI agent to turn on/off lights based on current time. 

In [25]:
var input = "Turn on the lights if it is night time.";

ChatHistory history = [];

// Get chat completion service
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

    // Get user input
Console.WriteLine($"User > {input}");
history.AddUserMessage(input);

// Enable auto function calling
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

// Get the response from the AI
var result = await chatCompletionService.GetChatMessageContentAsync(
        history,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel);

// Add the message from the agent to the chat history
history.AddMessage(result.Role, result.Content);

// Print the results
Console.WriteLine("Assistant > " + result);


User > Turn on the lights if it is night time.
Assistant > The lights are already on, and it is currently 15:00 (3:00 PM). There's no need to change the light setting at the moment.


If everything is set up correctly, we should see the output tells us the dummy lighs are turned on/off. 

## What to see here
<font >
Even the user only made one request, Semantic Kernel coordinated the whole process until task completed.

Semantic Kernel sent the prompt of user request with function metadata which tells AI service what functions are available to use.

Response from the AI service contained function names and argument the AI service chose to do the task. 

Semantic Kernel parsed the response, called functions with argument, and chained the function executions.
</font> 

In [20]:
    Console.ForegroundColor = ConsoleColor.DarkBlue;
    Console.WriteLine($"Chat History\n\n");
      
    for (var i = 0; i < history.Count; i++)
    {
        ChatMessageContent msg = history[i];
        
        if (msg.Role.ToString() == "tool") {
            Console.WriteLine($"{i} [{msg.Role}]");
            Console.WriteLine($"\t{msg.Metadata?["ChatCompletionsToolCall.Id"]}( {msg.Content} )\n");
        } else {
            Console.WriteLine($"{i} [{msg.Role}]:\n\t{msg.Content ?? " Null "} \n");
        }

        if (msg is OpenAIChatMessageContent) {
            OpenAIChatMessageContent aiMsg = (OpenAIChatMessageContent)msg;
            Console.WriteLine($"function calls: \n\t{aiMsg.Metadata?["ChatResponseMessage.FunctionToolCalls"]?.AsJson()} \n");
        }
    } 
 Console.ResetColor();

Chat History


0 [user]:
	Turn on the lights if it is night time. 

1 [assistant]:
	 Null  

function calls: 
	[
  {
    "Name": "TimePlugin-HourNumber",
    "Arguments": "{}",
    "Id": "call_qj2GVVu1J4U0r96dX0c5rXUX"
  },
  {
    "Name": "LightPlugin-GetState",
    "Arguments": "{}",
    "Id": "call_NaR93z5jMLBbFt8C4g8xRnOz"
  }
] 

2 [tool]
	call_qj2GVVu1J4U0r96dX0c5rXUX( 11 )

3 [tool]
	call_NaR93z5jMLBbFt8C4g8xRnOz( off )

4 [assistant]:
	 Null  

function calls: 
	[
  {
    "Name": "LightPlugin-ChangeState",
    "Arguments": "{\u0022newState\u0022:true}",
    "Id": "call_Z27B4gkMNyOCMrUpkRNzyke0"
  }
] 

5 [tool]
	call_Z27B4gkMNyOCMrUpkRNzyke0( on )

6 [assistant]:
	I've turned on the lights for you, as it is nighttime. 



## Wrapping up

  Semantic Kernel abstracts away complex AI service integration logic and provides a uniform interface to communicate with different AI providers.

  With its plugin system, we can extend AI cabilities by calling external tools.

  Semantic Kernel also manages the history and orchestrates function executions.